In [ ]:
#import libraries

import pandas as pd
import pyodbc, os
import requests
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#keys
API_KEY = 'xxxxx'
CHANNEL_ID = "xxxxx"

In [ ]:
#function to get video statistics

def get_video_details(video_id):

  url_video_stats = 'https://www.googleapis.com/youtube/v3/videos?id='+video_id+'&part=statistics&key='+API_KEY
  response_video_stats = requests.get(url_video_stats).json()

  view_count = response_video_stats['items'][0]['statistics']['viewCount']
  like_count = response_video_stats['items'][0]['statistics']['likeCount']
  comment_count = response_video_stats['items'][0]['statistics']['commentCount']

  return view_count,like_count,comment_count

In [ ]:
#function to get all videos from youtube channel and save in dataframe

def get_videos(df):
    #make API call
    pageToken = ''
    while 1:
        url = 'https://www.googleapis.com/youtube/v3/search?key='+API_KEY+'&channel_id='+CHANNEL_ID+'&part=snippet,id&order=date&maxResults=10000&'+pageToken
        response = requests.get(url).json()
  
        time.sleep(1) #give it a second before starting the for loop
        
        for video in response['items']:
            if video['id']['kind'] == 'youtube#video':
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace('&amp;','')   #can delete this line if title doesn't consist &amp
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split('T')[0]    #split data from timestamp
                
                view_count,like_count,comment_count = get_video_details(video_id)
      

      # save data to df
                df = df.append({'video_id':video_id,'video_title':video_title,
                       'upload_date':upload_date,'view_count':view_count,
                       'like_count':like_count,'comment_count':comment_count},ignore_index=True)
    
        try:
            if response['nextPageToken'] != None:               #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break
    
    return df
  


In [ ]:
#create dataframe to save the response from API call
df = pd.DataFrame(columns=['video_id','video_title','upload_date','view_count','like_count','comment_count'])

#make the call
df = get_videos(df)

In [ ]:
#function to establish connection with SQL SERVER

def connect_db(driver,server,db):
    try:
        conn = pyodbc.connect(f'Driver={driver};'
                      f'Server={server};'
                      f'Database={db};'
                      'Trusted_Connection=yes;')
    except InterfaceError:
        raise InterfaceError
        
    except OperationalError:
        raise OperationalError
        
    except ProgrammingError:
        raise ProgrammingError
        
    else:
        print('connected')
        
    return conn


In [ ]:
#function to create table in database
def create_table(curr):

    create_tbl_command = """
    IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[youtube_api]') AND type = N'U')
    BEGIN
        CREATE TABLE [dbo].[youtube_api] (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL )
    END """
    
    curr.execute(create_tbl_command)


#function to insert values to table in SQL SERVER database
def insert_into_table(curr, df):
    
    insert_into_videos = """INSERT INTO youtube_api (video_id, video_title, upload_date, view_count, like_count, comment_count)
                VALUES (?,?,?,?,?,?)"""
    
    row_to_insert = df.values.tolist()
    
    curr.executemany(insert_into_videos, row_to_insert)


#function to update the values if videos already exists
def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = """UPDATE youtube_api
            SET video_title = ?,
                view_count = ?,
                like_count = ?,
                comment_count = ?
            WHERE video_id = ?;"""
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)


#function to check if video exists or not
def check_if_video_exists(curr,video_id):
    query = '''SELECT video_id FROM youtube_api WHERE video_id = ?'''
    curr.execute(query,(video_id,))
    
    return curr.fetchone() is not None


#updating the database
#handling data for scalability

def update_db(curr,df):

    #temporary dataframe to store new videos 
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                   'like_count', 'comment_count'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']):    # If video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['comment_count'])

        else:    # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

In [ ]:
# connect to SQL SERVER database
driver = 'SQL Server'
server = 'HP\SQLEXPRESS'
db = 'youtube'
conn = None

conn = connect_db(driver,server,db)
curr = conn.cursor()

In [ ]:
#create table
create_table(curr)

In [ ]:
#store new values and videos
new_video_df = update_db(curr, df)
conn.commit()

In [ ]:
#insert into table
insert_into_table(curr, new_video_df)
conn.commit()
conn.close()

In [ ]:
#view data in db table

'''
curr.execute("SELECT * FROM youtube_api")
print(curr.fetchall())
'''